In [1]:
import json
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [2]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
import hashlib

In [4]:
def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc["id"] = generate_document_id(doc)

In [6]:
from collections import defaultdict

In [7]:
hashes = defaultdict(list)

In [8]:
for doc in documents:
    doc_id = doc["id"]
    hashes[doc_id].append(doc)

In [9]:
len(hashes), len(documents)

(947, 948)

In [10]:
for k, values in hashes.items():
    if len(values)>1:
        print(k, len(values))

593f7569 2


In [11]:
hashes["593f7569"]

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [12]:
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [13]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [14]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [20]:
from openai import OpenAI
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama'
)

In [21]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    json_response = response.choices[0].message.content
    return json_response



In [25]:
from tqdm.auto import tqdm

In [26]:
results = {}

In [ ]:
for doc in documents:
    doc_id = doc['id']
    if doc_id in results:
        continue
    questions = generate_questions(doc)
    results[doc_id] = questions

In [30]:
import pickle

In [31]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [36]:
results['58c9f99f']

'[\n"How can I resolve the Docker error \'invalid mode: \\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?",\n"What should I do if I encounter an invalid mode error in Docker on Windows?",\n"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",\n"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",\n"How do I correct the mounting path error in Docker for \\\\\\Program Files\\\\Git\\\\var\\\\lib\\\\postgresql\\\\data\'?"\n]'

In [37]:
parsed_results = {}

for doc_id, json_questions in results.items():
    try:
        parsed_results[doc_id] = json.loads(json_questions)
    except:
        pass

In [39]:
doc_index = {d['id']: d for d in documents}

In [41]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [43]:
import pandas as pd

In [44]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [45]:
df

,question,course,document
0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
...,...,...,...
4617,How should I destroy infrastructure created us...,mlops-zoomcamp,886d1617
4618,What is the first step to destroy AWS infrastr...,mlops-zoomcamp,886d1617
4619,Can I destroy infrastructure created with GitH...,mlops-zoomcamp,886d1617
4620,What command initializes Terraform with specif...,mlops-zoomcamp,886d1617


In [46]:
df.to_csv('ground-truth-data.csv', index=False)

In [47]:
!head ground-truth-data.csv


question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
